# Database and table creation

In this notebook, you will explore the best practices for IBM Db2 Event Store. You will learn:
- Database creation with IBM Db2 Event Store
- Best practices for table definition
- Best practices for indexing a table
- How to insert data from a CSV file

## Connect to IBM Db2 Event Store

### Determine the IP address of your host

Obtain the IP address of the host that you want to connect to by running the appropriate command for your operating system:

* On Mac, run: `ifconfig`
* On Windows, run: `ipconfig`
* On Linux, run: `hostname -i`

Edit the `HOST = "XXX.XXX.XXX.XXX"` value in the next cell to provide the IP address.

In [ ]:
# Set your host IP address
HOST = "XXX.XXX.XXX.XXX"

# Port will be 1100 for version 1.1.2 or later (5555 for version 1.1.1)
PORT = "1100"

# Database name
DB_NAME = "TESTDB"

# Table name
TABLE_NAME = "IOT_TEMPERATURE"

## Import Python modules


In [ ]:
from eventstore.common import ConfigurationReader
from eventstore.oltp import EventContext
from eventstore.sql import EventSession
from pyspark.sql import SparkSession

## Connect to Event Store

In [ ]:
endpoint = HOST + ":" + PORT
print("Event Store connection endpoint:", endpoint)
ConfigurationReader.setConnectionEndpoints(endpoint)

To run Spark SQL queries, you must set up a Db2 Event Store Spark session. The EventSession class extends the optimizer of the SparkSession class.

In [ ]:
sparkSession = SparkSession.builder.appName("EventStore SQL in Python").getOrCreate()
eventSession = EventSession(sparkSession.sparkContext, DB_NAME)

## Create the database
Run the following cell to try to create the database.

> Only one database can be active in Event Store Developer Edition. If you already have a database, you don't need to create one. To create a database in Event Store, you can use the createDatabase function. If you want to drop an existing database to create a new one, use the dropDatabase function first.

In [ ]:
# Run this cell if you need to (DROP and/or) CREATE the database.

# EventContext.drop_database(DB_NAME)   # Uncomment this if you want to drop an existing database
EventContext.create_database(DB_NAME)   # Comment this out (or skip this cell) to re-use an existing database

Now you can execute the command to open the database in the event session you created:

In [ ]:
eventSession.open_database()

## Explore the database by retrieving all table names

The following cell retrieves and prints the names of all tables in the database.
Run it now. You can come back and run it again after you create a table.

In [ ]:
with EventContext.get_event_context(DB_NAME) as ctx:
   print("Event context successfully retrieved.")

print("Table names:")
table_names = ctx.get_names_of_tables()
for name in table_names:
   print(name)

## Create a table with an index

The next cell defines the schema for the table we want to create.

In [ ]:
from eventstore.catalog import TableSchema
from pyspark.sql.types import *

tabSchema = TableSchema(TABLE_NAME, StructType([
    StructField("deviceID", IntegerType(), nullable = False),
    StructField("sensorID", IntegerType(), nullable = False),
    StructField("ts", LongType(), nullable = False),
    StructField("ambient_temp", DoubleType(), nullable = False),
    StructField("power", DoubleType(), nullable = False),
    StructField("temperature", DoubleType(), nullable = False)
    ]),
    sharding_columns = ["deviceID", "sensorID"],
    pk_columns = ["deviceID", "sensorID", "ts"]
                       )

The following cell defines the index schema which includes two equality columns -- *deviceID* and *sensorId*. The entries are sorted by *timestamp* in descending order. The *temperature* column is included to speed up queries that retrieve temperature:

In [ ]:
from eventstore.catalog import IndexSpecification, SortSpecification, ColumnOrder

indexSchema = IndexSpecification(
          index_name=TABLE_NAME + "Index",
          table_schema=tabSchema,
          equal_columns = ["deviceID", "sensorID"],
          sort_columns = [
            SortSpecification("ts", ColumnOrder.DESCENDING_NULLS_LAST)],
          include_columns = ["temperature"]
        )

Finally, the following cell is used to create the table with the index using the `create_table_with_index()` method, passing both the table schema and the index schema defined above:

In [ ]:
with EventContext.get_event_context(DB_NAME) as ctx:
   res = ctx.create_table_with_index(tabSchema, indexSchema)
   print("Table names:")
   table_names = ctx.get_names_of_tables()
   for name in table_names:
      print(name)

To drop a table we use the drop_table command, like in the cell below, but it is commented out and provided here only as a reference.

In [ ]:
# with EventContext.get_event_context(DB_NAME) as ctx:
#     ctx.drop_table(TABLE_NAME)

## Summary
In this notebook, you learned how to:
- Connect to IBM Db2 Event Store
- Create a new database
- Open a database
- Define a table schema and an index schema
- Create a database table with an index
- List the tables in a database

<p><font size=-1 color=gray>
&copy; Copyright 2019 IBM Corp. All Rights Reserved.
<p>
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file
except in compliance with the License. You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.
</font></p>